In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, when, to_timestamp, year, row_number
)
from pyspark.sql.window import Window


#initialize spark session
spark = SparkSession \
    .builder \
    .appName("CrimeDataAnalysis") \
    .getOrCreate()


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2736,application_1732639283265_2695,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:

import time

def process_data_with_dataframe_per_dataset(s3_link,csv = True):
     #read csv or parquet of the crime data
    if csv:
        crime_data_df = spark.read.csv(s3_link, header=True, inferSchema=True)
    else:
        crime_data_df = spark.read.parquet(s3_link, header=True, inferSchema=True)

    #add a year column by extracting the year from the date column
    crime_data_df = crime_data_df.withColumn(
        "year",
        year(to_timestamp(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a"))
    )
    #remove the mission area name as it doesn't match any of the precincts
    #(probably referncing joined operation of many precincts that we cannot include in the data)
    crime_data_df = crime_data_df.filter(col("AREA NAME") != "Mission")


    # calculate the closed case rate by year and by area(precinct)
    stats_df = crime_data_df.groupBy("year", "AREA NAME") \
        .agg(
            count("*").alias("total_crimes"),
            count(when((col("Status") != "IC") & (col("Status") != "UNK"), 1)).alias("solved_crimes")
        ) \
        .withColumn("closed_case_rate", (col("solved_crimes") / col("total_crimes")) * 100)

    # define a window for ranking areas by closed case rate within each year
    window_spec = Window.partitionBy("year").orderBy(col("closed_case_rate").desc())
    #add the column of the rank
    stats_df = stats_df.withColumn("#", row_number().over(window_spec))

    # keep the 3 ares with the highesst rate
    top_3_df = stats_df.filter(col("#") <= 3)

    # Return the processed dataFrame with the selected columns
    return top_3_df.select(
        col("year"),
        col("AREA NAME").alias("precinct"),
        col("closed_case_rate"),
        col("#")
    ).orderBy("year", "#")





FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Process data for 2010s and 2020s datasets
start_time_df = time.time()

top3_2010s = process_data_with_dataframe_per_dataset(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
)

top3_2020s = process_data_with_dataframe_per_dataset(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
)

# Combine the two datasets and sort by year and rank
combined_top3_df = top3_2010s.union(top3_2020s).orderBy("year", "#")
end_time_df = time.time()
execution_time_df = end_time_df - start_time_df

# Show the combined result
combined_top3_df.show(100, truncate=False)
print(f"Total execution time (DataFrame): {execution_time_df:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+---+
|year|precinct   |closed_case_rate  |#  |
+----+-----------+------------------+---+
|2010|Rampart    |32.84713448949121 |1  |
|2010|Olympic    |31.515289821999087|2  |
|2010|Harbor     |29.36028339237341 |3  |
|2011|Olympic    |35.040060090135206|1  |
|2011|Rampart    |32.4964471814306  |2  |
|2011|Harbor     |28.51336246316431 |3  |
|2012|Olympic    |34.29708533302119 |1  |
|2012|Rampart    |32.46000463714352 |2  |
|2012|Harbor     |29.509585848956675|3  |
|2013|Olympic    |33.58217940999398 |1  |
|2013|Rampart    |32.1060382916053  |2  |
|2013|Harbor     |29.723638951488557|3  |
|2014|Van Nuys   |32.0215235281705  |1  |
|2014|West Valley|31.49754809505847 |2  |
|2014|Foothill   |30.723059743160245|3  |
|2015|Van Nuys   |32.265140677157845|1  |
|2015|Foothill   |30.353001803658852|2  |
|2015|West Valley|30.316223648029332|3  |
|2016|Van Nuys   |32.194518462124094|1  |
|2016|West Valley|31.40146437042384 |2  |
|2016|Foothill   |29.9086472281316

In [3]:
group_number = "1"
s3_path = "s3://groups-bucket-dblab-905418150721/group"+group_number+"/top-precinct-closed-case-df/"
combined_top3_df.write.mode("overwrite").parquet(s3_path)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
import time


def process_data_with_sql_api(spark, s3_link, view_name,csv = True):
  
    #read csv or parquet of the crime data

    if csv:
        crime_data_df = spark.read.csv(s3_link, header=True, inferSchema=True)
    else:
        crime_data_df = spark.read.parquet(s3_link)
    
    # create a  view for sql queries
    crime_data_df.createOrReplaceTempView(view_name)

    
    query = f"""
    WITH InitialData AS (
        SELECT
            *,
            YEAR(TO_TIMESTAMP(`DATE OCC`, 'MM/dd/yyyy hh:mm:ss a')) AS year
        FROM
            {view_name}

    ),
    GrouppedData AS (
        SELECT
            year,
            `AREA NAME` AS precinct,
            (COUNT(CASE WHEN Status NOT IN ('IC', 'UNK') THEN 1 END) * 100.0) / COUNT(*) AS closed_case_rate
        FROM
            InitialData
        WHERE
            `AREA NAME` != 'Mission'
        GROUP BY
            year,
            `AREA NAME`
    ),
    RankedData AS (
        SELECT
            *,
            ROW_NUMBER() OVER (PARTITION BY year ORDER BY closed_case_rate DESC) AS rank
        FROM
            GrouppedData
    )
    SELECT
        year,
        precinct,
        closed_case_rate,
        rank AS `#`
    FROM
        RankedData
    WHERE
        rank <= 3
    ORDER BY
        year,
        rank
    """
    
    # execute the SQL query
    top3_df = spark.sql(query)
    
    return top3_df


spark = SparkSession \
    .builder \
    .appName("CrimeDataAnalysisSQL") \
    .getOrCreate()    
   
s3_link_2010s = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
s3_link_2020s = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"

start_time = time.time()
       
# process the 2010s dataset
top3_2010s = process_data_with_sql_api(
        spark,
        s3_link_2010s,
        view_name="crime_data_2010s_temp_view"
)
        
# process the 2020s dataset
top3_2020s = process_data_with_sql_api(
        spark,
        s3_link_2020s,
        view_name="crime_data_2020s_temp_view"
)
        
# create temporary views for the top3 DataFrames
top3_2010s.createOrReplaceTempView("top3_2010s_view")
top3_2020s.createOrReplaceTempView("top3_2020s_view")

# SQL query to combine the results
combined_query = """
SELECT * FROM top3_2010s_view
UNION ALL
SELECT * FROM top3_2020s_view
ORDER BY year, `#`
"""

# execute the combined SQL query
combined_top3_df = spark.sql(combined_query)
end_time = time.time()
execution_time = end_time - start_time
        
# showcase the  results
combined_top3_df.show(100, truncate=False)

print(f"Total execution time (SQL): {execution_time:.2f} seconds")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+-----------------+---+
|year|precinct   |closed_case_rate |#  |
+----+-----------+-----------------+---+
|2010|Rampart    |32.84713448949121|1  |
|2010|Olympic    |31.51528982199909|2  |
|2010|Harbor     |29.36028339237341|3  |
|2011|Olympic    |35.04006009013520|1  |
|2011|Rampart    |32.49644718143060|2  |
|2011|Harbor     |28.51336246316431|3  |
|2012|Olympic    |34.29708533302119|1  |
|2012|Rampart    |32.46000463714352|2  |
|2012|Harbor     |29.50958584895668|3  |
|2013|Olympic    |33.58217940999398|1  |
|2013|Rampart    |32.10603829160530|2  |
|2013|Harbor     |29.72363895148855|3  |
|2014|Van Nuys   |32.02152352817050|1  |
|2014|West Valley|31.49754809505847|2  |
|2014|Foothill   |30.72305974316025|3  |
|2015|Van Nuys   |32.26514067715784|1  |
|2015|Foothill   |30.35300180365885|2  |
|2015|West Valley|30.31622364802933|3  |
|2016|Van Nuys   |32.19451846212410|1  |
|2016|West Valley|31.40146437042384|2  |
|2016|Foothill   |29.90864722813165|3  |
|2017|Van Nuys  

In [6]:
#save the results
group_number = "1"
s3_path = "s3://groups-bucket-dblab-905418150721/group"+group_number+"/top-precinct-closed-case-sql/"
combined_top3_df.write.mode("overwrite").parquet(s3_path)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from pyspark.sql import SparkSession
#create parquet files from the CSVs
spark = SparkSession.builder.appName("CSV to Parquet").getOrCreate()
#csv paths
s3_link_2010s = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
s3_link_2020s = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
group_number = "1"
#parquet paths
s3_path_2010s = "s3://groups-bucket-dblab-905418150721/group"+group_number+"/parquet-files/crime2010s.parquet"
s3_path_2020s = "s3://groups-bucket-dblab-905418150721/group"+group_number+"/parquet-files/crime2020s.parquet"
#load the data
df_2010s = spark.read.csv(s3_link_2010s, header=True, inferSchema=True)
df_2020s = spark.read.csv(s3_link_2020s, header=True, inferSchema=True)
#write the parquet files
df_2010s.write.mode("overwrite").parquet(s3_path_2010s_parquet)
df_2020s.write.mode("overwrite").parquet(s3_path_2020s_parquet)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession \
    .builder \
    .appName("CrimeDataAnalysisDF") \
    .getOrCreate()    

start_time = time.time()   
group_number = "1"
s3_path_2010s = "s3://groups-bucket-dblab-905418150721/group"+group_number+"/parquet-files/crime2010s.parquet"
s3_path_2020s = "s3://groups-bucket-dblab-905418150721/group"+group_number+"/parquet-files/crime2020s.parquet"


       
# process the 2010s dataset
top3_2010s = process_data_with_dataframe_per_dataset(
        s3_path_2010s,
        csv=False
)
        
# process the 2020s dataset
top3_2020s = process_data_with_dataframe_per_dataset(
        s3_path_2020s,
        csv=False
)
        
# combine the results from both datasets
combined_top3_df = top3_2010s.union(top3_2020s).orderBy("year", "#")
end_time = time.time()
execution_time = end_time - start_time
        
# shοw the results
combined_top3_df.show(100, truncate=False)

print(f"Total execution time parquet (Dataframe): {execution_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+---+
|year|precinct   |closed_case_rate  |#  |
+----+-----------+------------------+---+
|2010|Rampart    |32.84713448949121 |1  |
|2010|Olympic    |31.515289821999087|2  |
|2010|Harbor     |29.36028339237341 |3  |
|2011|Olympic    |35.040060090135206|1  |
|2011|Rampart    |32.4964471814306  |2  |
|2011|Harbor     |28.51336246316431 |3  |
|2012|Olympic    |34.29708533302119 |1  |
|2012|Rampart    |32.46000463714352 |2  |
|2012|Harbor     |29.509585848956675|3  |
|2013|Olympic    |33.58217940999398 |1  |
|2013|Rampart    |32.1060382916053  |2  |
|2013|Harbor     |29.723638951488557|3  |
|2014|Van Nuys   |32.0215235281705  |1  |
|2014|West Valley|31.49754809505847 |2  |
|2014|Foothill   |30.723059743160245|3  |
|2015|Van Nuys   |32.265140677157845|1  |
|2015|Foothill   |30.353001803658852|2  |
|2015|West Valley|30.316223648029332|3  |
|2016|Van Nuys   |32.194518462124094|1  |
|2016|West Valley|31.40146437042384 |2  |
|2016|Foothill   |29.9086472281316